<a href="https://colab.research.google.com/github/RainXie23/MANE4961---Machine-Learning-for-Engineering-Fall-2025-/blob/main/MLE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
url = "https://www.fpbase.org/api/proteins/?format=csv"
df = pd.read_csv(url)
df.to_csv("fpbase_with_sequences.csv", index=False)
print(df.columns)

Index(['agg', 'doi', 'genbank', 'ipg_id', 'name', 'pdb', 'pdb.0', 'pdb.1',
       'pdb.10', 'pdb.2',
       ...
       'transitions.2.to_state', 'transitions.2.trans_wave',
       'transitions.3.from_state', 'transitions.3.to_state',
       'transitions.3.trans_wave', 'transitions.4.from_state',
       'transitions.4.to_state', 'transitions.4.trans_wave', 'uniprot',
       'uuid'],
      dtype='object', length=117)


In [22]:
!pip install biopython
from Bio.SeqUtils import molecular_weight

df_no_ambiguous = df[df['seq'].apply(lambda x: ('X' not in x and 'Z' not in x and 'B' not in x) if isinstance(x, str) else False)]
df_no_ambiguous['mol_weight'] = df_no_ambiguous['seq'].apply(lambda s: molecular_weight(s, seq_type='protein') if isinstance(s, str) else None)

/tmp/ipython-input-815984273.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_ambiguous['mol_weight'] = df_no_ambiguous['seq'].apply(lambda s: molecular_weight(s, seq_type='protein') if isinstance(s, str) else None)


In [26]:
df = df_no_ambiguous[
    (df_no_ambiguous['switch_type'] == 'b') &
    df_no_ambiguous['agg'].notna() &
    (df_no_ambiguous['agg'] != '') &
    df_no_ambiguous['states.0.em_max'].notna() &
    df_no_ambiguous['states.0.brightness'].notna()
]

len(df['name'])

435

In [34]:
import collections
import numpy as np

amino_acids_list = list('ARNDCQEGHILKMFPSTWYV')
aaindex = {aa: i for i, aa in enumerate(amino_acids_list)}

def Seq2OneHot(seq, amino_acid_alphabet):
  max_len = max(len(s) for s in df['seq'])
  one_hot_seq = np.zeros((max_len, len(amino_acid_alphabet)))
  for i, char in enumerate(seq):
    if i < max_len and char in amino_acid_alphabet:
      one_hot_seq[i, aaindex[char]] = 1
  return one_hot_seq

def Position(seq, amino_acid_alphabet):
  most_aa_in_seq = 0
  if isinstance(seq, str) and len(seq) > 0:
    counts = collections.Counter(seq)
    if counts:
      most_aa_in_seq = max(counts.values())

  # Ensure most_aa is at least 1 to avoid issues with empty sequences or zero counts
  if most_aa_in_seq == 0 and len(seq) > 0: # If there are AAs but counts are all zero, it's an issue
      most_aa_in_seq = 1 # Smallest possible dimension for position if seq is not empty
  elif len(seq) == 0:
      return np.zeros((len(amino_acid_alphabet), 0)) # Return empty array if sequence is empty

  position = np.zeros((len(amino_acid_alphabet), most_aa_in_seq))
  aa_occurrence_count = {aa: 0 for aa in amino_acid_alphabet}

  for i, char_in_seq in enumerate(seq):
    if char_in_seq in aaindex:
      aa_index = aaindex[char_in_seq]
      occurrence_idx = aa_occurrence_count[char_in_seq]
      if occurrence_idx < most_aa_in_seq:
          position[aa_index, occurrence_idx] = 1
      aa_occurrence_count[char_in_seq] += 1
  return position

In [35]:
df['seq_one_hot'] = df['seq'].apply(lambda s: Seq2OneHot(s, amino_acids_list))
df['seq_position'] = df['seq'].apply(lambda s: Position(s, amino_acids_list))

/tmp/ipython-input-305362229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['seq_one_hot'] = df['seq'].apply(lambda s: Seq2OneHot(s, amino_acids_list))
/tmp/ipython-input-305362229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['seq_position'] = df['seq'].apply(lambda s: Position(s, amino_acids_list))


In [ ]:
def Agg2Num(agg)
  if agg == 'm'
    return 1
  elif agg == 'd' | 'wd'
    return 2
  elif agg == 'td'
    return 2.5
  elif agg == 't'
    return 4

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV



x_train, x_test, y_train, y_test = train_test_split(df[]